In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
pip install scikit-image

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install graphviz


Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install dask-image

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for dask-image: filename=dask_image-2022.9.0-py2.py3-none-any.whl size=41053 sha256=993ba6518bb789b897fa68dc0c6aaa8a386a0acdd29af57924296371f220fe76
  Stored in directory: /root/.cache/pip/wheels/e6/37/fe/ca89bb44d2f40604840d754ae0fe1f597b9cdb32c3fdac141f
  Created wheel for pims: filename=PIMS-0.6.1-py3-none-any.whl size=82635 sha256=08472410c1356fb91880b0f201d59cd867574a5659099de1e7d15cd3e7f43782
  Stored in directory: /root/.cache/pip/wheels/88/36/0c/b3160c75edaf29447bc41f530cca51c0c25974334f3fedc0ed
Successfully built dask-image pims
Note: you may need to restart the kernel to use updated packages.


In [7]:

pip install imageio[pyav]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 37.4 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
#In console run following:
#pip install scikit-image
#pip install graphviz
#pip install dask-image
#pip install imageio[pyav]
import dask
import matplotlib
import numpy
import dask_image.imread
import dask_image.ndfilters
import dask_image.ndmeasure
import dask.array as da
import glob
import re
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
labels = []
include50labels = ['Bird','Black','Car','Dog','Fall','Father','Good Morning', 'Red','Summer','White','loud','quiet','happy','long','short','big large', 'small little','hot', 'new','dry','good','Cow','Hat', 'T-Shirt', 'Shoes', 'Monday', 'Year', 'Time', 'Fan', 'Cell phone', 'Hello', 'Thank you', 'Window', 'Pen', 'Paint', 'Teacher', 'Priest', 'train ticket', 'Brother', 'Boy', ' Girl', ' Bank', 'I', 'it','you', 'Election', 'Death', 'Court', 'House', 'Store or Shop']


In [ ]:
def return_label(dirname):
    return re.sub('\d+. ','',dirname)
datasets = ['include-50','include-50-2']
for dataset in datasets:
    os.chdir("/kaggle/input/{dataset_name}".format(dataset_name = dataset))
    root_dir_list = os.listdir()
    for root_dir in root_dir_list:
        os.chdir(os.path.join(os.getcwd(),root_dir))
        dir_list = os.listdir()
        for dir in dir_list:
            os.chdir(os.path.join(os.getcwd(),dir))
            n = len(os.listdir())
            label = return_label(dir)
            for i in range(n):
                labels.append(label)
            os.chdir('../')
        os.chdir('../')

In [ ]:
extra_labels= { 'Bird':4,'Bank':1, 'Store or Shop':1,'Court':2}
def insert_extra_labels(labels, extra_labels):
    for extra_label in extra_labels:
        index = labels.index(extra_label)
        for i in range(extra_labels[extra_label]):
            labels.insert(index,extra_label)
    return labels

In [ ]:
labels = insert_extra_labels(labels,extra_labels)

In [ ]:
len(labels)

In [ ]:
video_frames = []
datasets = ['include-50','include-50-2']
for dataset in datasets:
    for root,_,filenames in os.walk("/kaggle/input/{dataset_name}".format(dataset_name = dataset)):
        for filename in filenames:
            #print(os.path.join(root,filename))
            file_path = os.path.join(root,filename)
            x = dask_image.imread.imread(file_path)
            video_frames.append(x)

In [ ]:
len(video_frames)

In [ ]:
%cd /kaggle/working
%ls

In [ ]:
rm -r -d pytorch-openpose
ls

In [ ]:

!git clone https://github.com/Hzzone/pytorch-openpose
%cd pytorch-openpose

In [ ]:
# import sys
# sys.path.append(os.path.dirname(os.path.join(os.getcwd(),"src")))

In [ ]:
import matplotlib.pyplot as plt
import copy
from src import model
from src import util
from src.body import Body
from src.hand import Hand


In [ ]:

!cp /kaggle/input/openpose-pretrained-models/body_pose_model.pth model
!cp /kaggle/input/openpose-pretrained-models/hand_pose_model.pth model


In [ ]:

body_estimation = Body('model/body_pose_model.pth')
hand_estimation = Hand('model/hand_pose_model.pth')

In [ ]:
def get_all_hand_peaks(hands_list,oriImg):
    all_hand_peaks = []
    for x, y, w, is_left in hands_list:
        peaks = hand_estimation(oriImg[y:y+w, x:x+w, :]) #pretrained model, last layer: convolution_param-> num_output: 22, 21 Keypoints
        #peaks is coordinates wrt x,y top left.
        #adjust coordinates of keypoints
        peaks[:, 0] = np.where(peaks[:, 0]==0, peaks[:, 0], peaks[:, 0]+x)
        peaks[:, 1] = np.where(peaks[:, 1]==0, peaks[:, 1], peaks[:, 1]+y) 
        all_hand_peaks.append(peaks)
    return all_hand_peaks
def generate_hand_keypoints_for_one_video(video):
    vdo_keypoints = []
    Num_of_frames = len(video)
    for f in range(Num_of_frames):
        try:
            oriImg = np.array(video[f])
        except:
            continue
        if(oriImg is None):
            continue
        candidate, subset = body_estimation(oriImg)
        canvas = copy.deepcopy(oriImg)
        hands_list = util.handDetect(candidate, subset, oriImg)
        all_hand_peaks = get_all_hand_peaks(hands_list,oriImg)
        vdo_keypoints.append(all_hand_peaks)
    return vdo_keypoints
vdo_keypoints_list = []
def generate_hand_keypoints_for_multiple_video(video_list,start_range, end_range):
    #(start_range, last_range], including start and excluding last.
    for i in range(start_range, end_range):
        print(i)
        vdo_keypoints_list.append(generate_hand_keypoints_for_one_video(video_list[i]))


In [ ]:
len(vdo_keypoints_list)
#num_of_vdosxnum_of_framesx2_handsx21_keypointsx2_cordinate(x,y)


In [ ]:
#if vdo_keypoints contains keypoints from ur last run, please make sure u select appropriate range
df = pd.DataFrame({'Label':labels[1015:1020],'Keypoints':vdo_keypoints_list})

In [ ]:
from IPython.display import FileLink
FileLink(r'Video_keypoints_five.csv')

In [ ]:
start = 1030
end = 1035
generate_hand_keypoints_for_multiple_video(video_frames, start,end)
df = pd.DataFrame({'Label':labels[start:end],'Keypoints':vdo_keypoints_list})
%cd /kaggle/working
df.to_csv('Video_keypoints_onethousandthirty.csv')

In [ ]:
ls


In [ ]:
df